In [1]:
from requests import get
from bs4 import BeautifulSoup

In [2]:
# https://shop.flixbus.pl/search?departureCity=7568&arrivalCity=1915&route=Warszawa-Krak%C3%B3w&rideDate=20.11.2019&adult=1&_locale=pl&wt_eid=2156377373864875860&wt_t=1573237819472&affiliate=(not%2Bset)%2C&_ga=2.228166996.854006483.1573236402-171244053.1566906410&atb_pdid=1524aa7b-e10f-4744-aa6b-c1c033b32c8b&_sp=37e78191-3492-43e3-ab42-1cf0bcb5005a
search_params = {
    "departureCity":"7568",
    "arrivalCity":"1915",
    "route":"Warszawa-Kraków",
    "rideDate":"20.11.2019",
    "adult":"1",
    "_locale":"pl",
}
request = get("https://shop.flixbus.pl/search?", params= search_params)

In [3]:
# dir(request)

print(request.status_code)
site = BeautifulSoup(request.content, "html.parser")

200


In [23]:
results = site.find("div", id = "results-group-container-direct")

In [24]:
len(results)

12

In [6]:
# class ride-avaliable - sprawdzić jak wygląda kiedy nie ma dostępnych
# Case z pokazywaniem jazd kolejnego dnia jest niobsłużony jeszcze
rows = results.find_all("div", class_ = "ride-available")

In [7]:
# jeden row odpowiada za jeden rezultat - teraz wyciągam wszysko po kolei
single_row = rows[0]

In [8]:
results_dict = {}

In [9]:
ride_times = single_row.find("div", class_ = "ride-times")
results_dict["departure"] = ride_times.find("div", class_ = "departure").text
results_dict["arrival"] = ride_times.find("div", class_ = "arrival").text

In [10]:
ride_stations = single_row.find("div", class_ = "ride-station-names")
results_dict["departure_station"] = ride_stations.find("div", class_ = "departure-station-name").text
results_dict["arrival_station"] = ride_stations.find("div", class_ = "arrival-station-name").text

In [11]:
results_dict

{'departure': '06:35',
 'arrival': '11:00',
 'departure_station': 'Warszawa Zachodnia',
 'arrival_station': 'Kraków MDA'}

In [12]:
# Długość jazdy - prawdopodobnie nie będzie potrzebne zbyt szybko więc nei pobieram
single_row.find("div", class_ = "ride__duration")

<div class="col-xs-12 duration ride__duration ride__duration-messages">
                                                4:25 <span class=" ride__duration-suffix duration-suffix">Godz.</span><span class="ride-message has-popup" data-popup-id="popup-ride-message-direct-99446941-10118-2196"><img alt="" src="https://d3gelgmyzspcuw.cloudfront.net/bf4785dfc/img/icons/info-icon@2x.png"/><div class="popup" id="popup-ride-message-direct-99446941-10118-2196"><span class="popup-top"><div>
                             System rozrywki pokładowej: filmy, audiobooki i gry za darmo na Twoim urządzeniu mobilnym
                        </div></span><span class="popup-bot"></span></div></span></div>

In [26]:
# Używam brzydkiego hacka do usunięcia końcówki ceny - zostawiam tylko część bez groszy
results_dict["price"] = single_row.find("span", class_ = "num currency-small-cents").text#.split(",")[0]


In [27]:
results_dict

{'departure': '06:35',
 'arrival': '11:00',
 'departure_station': 'Warszawa Zachodnia',
 'arrival_station': 'Kraków MDA',
 'price': '35,99\xa0zł'}

In [32]:
results_dict["price"].split("\xa0zł")[0]

'35,99'

In [15]:
# Czasami tutaj będą trzy wartosci, razem z walutą w dziwnym formacie. wtedy trzeba:
# a.children[2].extract()

Skrypt zwraca podstawowe informacje o danej trasie
Co dalej? Opakowanie w funkcję, przetestowanie na edge casach.
Edge casy:
- Nie ma ani jednej jazdy z danymi parametrami
- Niektóre jazdy wyszukane są kolejnego dnia
- Brak miejsc
- Wyciąganie info o pozostałych miejscach


Docelowo:

- Apka do scrapowania dla końcowych użytkowników
- Apka z intersection warunków
- Samo api?

Architektura do scrapowania:

#### Serwer scrapujący 
- chodzi cały czas, np AWS
- Można zdefiniować joby do scrapowania- dyskusyjne czy za pomocą api bo to dużo roboty
- Serwer odkłada dane do bazy po zescrapowaniu 
- 

#### Baza danych
- Np AWS
- Otrzymuje dane od serwera scrapującego i odkłada w tabelach
- 

#### Apka dla użytkownika
- Można zdefiniować przecięcia
- Pola: stacja początkowa, stacja końcowa, data, ilość osób (pojedyncze)
- Przycisk Generuj
- Można pobrać rezultaty do csv


#### Apka dla użytkownika - 2 pomysł
- Prosty dashboard, z którego można pobrać bazę danych 
- Logowanie dla admina


In [16]:
()

()